In [ ]:
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from pyserini.index import IndexReader
import pandas as pd
from pyserini.search import LuceneSearcher
from tqdm import tqdm


In [ ]:
import os
import pandas as pd
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from dask.distributed import Client, LocalCluster
from dask import dataframe as dd
import re


In [ ]:

nlp = English()
nlp.Defaults.stop_words = STOP_WORDS

def preprocess_text(text, remove_stopwords=False, apply_stemming=False):
    if not isinstance(text, str):
        text = str(text)
    # Convert to lowercase
    text = text.lower()
    
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    doc = nlp(text)
    if remove_stopwords and apply_stemming:
        return ' '.join([token.lemma_ for token in doc if not token.is_stop])
    elif remove_stopwords:
        return ' '.join([token.text for token in doc if not token.is_stop])
    elif apply_stemming:
        return ' '.join([token.lemma_ for token in doc])
    else:
        return ' '.join([token.text for token in doc])

def process_tsv_file(file_path, variant):
    # Load TSV file with Dask (replace 'text' with the name of the column containing the text)
    df = dd.read_csv(file_path, sep='\t', blocksize='64MB', header=None, names=['docno', 'URL','text','dn']) # adjust blocksize and column names as needed
    df = df[['docno', 'text', 'dn']]
    
    # Apply preprocessing
    df['text'] = df['text'].apply(lambda text: preprocess_text(text, remove_stopwords=variant['stopwords'], apply_stemming=variant['stemming']), meta=('text', 'str'))

    # Compute and return as pandas dataframe
    return df.compute()



# Initialize Dask client
if not 'client' in locals():
    cluster = LocalCluster(n_workers=4, threads_per_worker=1)
    client = Client(cluster)

# Preprocessing options
index_variants = [
    {
        "name": "full_index",
        "index_path": "full_index/",
        "stopwords": False,
        "stemming": False,
    },
    {
        "name": "stopwords_removed",
        "index_path": "stopwords_removed/",
        "stopwords": True,
        "stemming": False,
    },
    {
        "name": "stemming",
        "index_path": "stemming/",
        "stopwords": False,
        "stemming": True,
    },
    {
        "name": "stopwords_removed_stemming",
        "index_path": "stopwords_removed_stemming/",
        "stopwords": True,
        "stemming": True,
    },
]


path_to_tsv_file = 'data/msmarco-docs.tsv'



In [10]:
import os
import pandas as pd
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from dask.distributed import Client, LocalCluster
from dask import dataframe as dd
import re

nlp = English()
nlp.Defaults.stop_words = STOP_WORDS

def preprocess_text(text, remove_stopwords=False, apply_stemming=False):
    if not isinstance(text, str):
        text = str(text)
    # Convert to lowercase
    text = text.lower()
    
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    doc = nlp(text)
    if remove_stopwords and apply_stemming:
        return ' '.join([token.lemma_ for token in doc if not token.is_stop])
    elif remove_stopwords:
        return ' '.join([token.text for token in doc if not token.is_stop])
    elif apply_stemming:
        return ' '.join([token.lemma_ for token in doc])
    else:
        return ' '.join([token.text for token in doc])

def process_tsv_file(file_path, variant):
    # Load TSV file with Dask (replace 'text_column' with the name of the column containing the text)
    df = dd.read_csv(file_path, sep='\t', blocksize='64MB', header=None, names=['docno', 'URL','text_column','dn'],) # adjust blocksize and column names as needed
    # Apply preprocessing
    df['text_column'] = df['text_column'].apply(lambda text: preprocess_text(text, remove_stopwords=variant['stopwords'], apply_stemming=variant['stemming']), meta=('text_column', 'str'))

    # Save preprocessed data to a TSV file
    output_file = variant["index_path"] + 'preprocessed_*.tsv'
    df.to_csv(output_file, sep='\t', index=False, compute=True)

# Initialize Dask client
if not 'client' in locals():
    cluster = LocalCluster(n_workers=4, threads_per_worker=1)
    client = Client(cluster)

# Path to your TSV file
path_to_tsv_file = 'data/msmarco-docs.tsv'




In [17]:
# Preprocessing options
index_variants = [
    {
        "name": "full_index",
        "index_path": "full_index/",
        "stopwords": False,
        "stemming": False,
    },
    {
        "name": "stopwords_removed",
        "index_path": "stopwords_removed/",
        "stopwords": True,
        "stemming": False,
    },
    {
        "name": "stemming",
        "index_path": "stemming/",
        "stopwords": False,
        "stemming": True,
    },
    {
        "name": "stopwords_removed_stemming",
        "index_path": "stopwords_removed_stemming/",
        "stopwords": True,
        "stemming": True,
    },
]


In [18]:
full = index_variants[0]
stopwords_removed = index_variants[1] 
stemming = index_variants[2] 
stopwords_removed_stemming = index_variants[3] 


In [19]:
full_df = process_tsv_file(path_to_tsv_file, full
)


In [16]:
full_df.to_csv( 'full.tsv', sep='\t', index=False)
# full_df.to_csv(
# "."+ 'full.tsv', sep='\t', index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [ ]:
stopwords_removed_df = process_tsv_file(path_to_tsv_file, stopwords_removed
)
stopwords_removed_df.to_csv(
"."+ 'stopwords_removed.tsv', sep='\t', index=False)

In [ ]:
stemming_df = process_tsv_file(path_to_tsv_file, stemming)
stemming_df.to_csv("." + 'stemming.tsv', sep='\t', index=False)



In [ ]:
stopwords_removed_stemming_df = process_tsv_file(path_to_tsv_file, stopwords_removed_stemming)
stopwords_removed_stemming_df.to_csv("." + 'stopwords_removed_stemming.tsv', sep='\t', index=False)

In [ ]:
(pd.read_csv('full.tsv', sep='\t').head(2)['text_column']).str.split().apply(len).sum()